# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def common_correlation(cities, big4_df, common_df, replaceMap):
    
    
    cities.rename(columns={'Metropolitan area':'MA','Population (2016 est.)[8]':'Population'},inplace=True)
    df= cities
    #pd.set_option('display.max_rows', len(df))
    #print(df)
    regions=df['MA']
    regions = regions.str.replace('^(?P<ONE>.*)–.*$',lambda m: m.group('ONE')).str.replace('^(?P<ONE>.*), D\.C\.$',lambda m: m.group('ONE')).str.replace('^(?P<ONE>.*) Bay Area$',lambda m: m.group('ONE')).str.replace('^(?P<ONE>.*) Area$',lambda m: m.group('ONE')).str.replace('^(?P<ONE>.*) City$',lambda m: m.group('ONE'))

    df = big4_df
    df = df.replace(r'^\s*$', np.NaN, regex=True).replace(r'^.*—.*$', np.NaN, regex=True).replace(r'^$', np.NaN, regex=True)
    df = df.str.replace(r'^(?P<ONE>.*)\[.*\](?P<TWO>.*)$', lambda m: m.group('ONE') + m.group('TWO'), regex=True)
    #df = df.str.findall('([A-Z0-9][0-9a-z]+ [A-Z0-9][0-9a-z]+|[A-Z0-9][0-9a-z]+)').apply(lambda x:pd.Series(x))
    findf = df.str.findall('([A-Z0-9][0-9a-z]+ [A-Z0-9][0-9a-z]+|[A-Z0-9][0-9a-z]+)').apply(lambda x:pd.Series(x))
    
    tdf = cities
    tdf['regions']= regions
    
    
    #tdf[['team1','team2','team3']] = pd.DataFrame({'team1':df[0],'team2':df[1],'team3':df[2]})
    #df = tdf[['regions','Population','team1','team2','team3']]
    #df = pd.melt(df, id_vars=['regions','Population'], value_vars=['team1','team2','team3'], value_name='team')
    
    
    colname = ['team1','team2','team3']
    tdf[[colname[x] for x in findf]] = pd.DataFrame({x:findf[x] for x in findf})
    df= tdf[['regions','Population']]
    df[[colname[x] for x in findf]] = tdf[[colname[x] for x in findf]]
    df = pd.melt(df, id_vars=['regions','Population'], value_vars=[colname[x] for x in findf], value_name='team')
    
    
    df=df.dropna()
    df= df[['regions','Population','team']]
    df['team']= df['regions'] + ' ' + df['team'] 
    df['team'] = df['team'].replace(replaceMap)
    df['Population']=df['Population'].apply(pd.to_numeric)
    #pd.set_option('display.max_rows', len(df))
    #print(df)
    df1=df
    population_by_region = df.groupby(by='regions').mean()['Population']
    #print(population_by_region)

    repl = lambda m: m.group('ONE')
    df = common_df
    df = df[~ (df['team'].str.contains('Division') | (df['team'].str.contains('AFC ')) | (df['team'].str.contains('NFC '))) ][df['year']==2018]
    df[['W','L']] = df[['W','L']].apply(pd.to_numeric)
    df = pd.DataFrame({'team':df['team'].str.replace('^(?P<ONE>.*)\s\([0-9]+\)$',repl, regex=True).str.replace('^(?P<ONE>[^\*]+)\**$',repl, regex=True).str.replace('^(?P<ONE>[^\+]+)\+*$',repl, regex=True),'ratio':df['W']/(df['W']+ df['L'])})
    #pd.set_option('display.max_rows', len(df))
    #print(df)
    group_df = df1.merge(df, how='left', on='team').groupby(by='regions').mean()
    win_loss_by_region=group_df['ratio']
    #print(win_loss_by_region)

    FalseMatrix = win_loss_by_region.notnull()
    win_loss_by_region = win_loss_by_region[FalseMatrix].tolist()
    population_by_region = population_by_region[FalseMatrix].tolist()

    return population_by_region, win_loss_by_region



def nhl_correlation(): 
    # YOUR CODE HERE

    
    replaceMap = {'Las Vegas Golden Knights':'Vegas Golden Knights',
                                  'Tampa Lightning':'Tampa Bay Lightning',
                                  'Raleigh Hurricanes':'Carolina Hurricanes',
                                  'Miami Panthers':'Florida Panthers',
                                  'San Francisco Sharks':'San Jose Sharks',
                                  'Denver Avalanche':'Colorado Avalanche',
                                  'Minneapolis Wild':'Minnesota Wild',
                                  'Phoenix Coyotes':'Arizona Coyotes',
                                  'Los Angeles Ducks':'Anaheim Ducks',
                                  'New York Devils':'New Jersey Devils'
                                 }
    #
    population_by_region, win_loss_by_region = common_correlation(cities, cities['NHL'], nhl_df, replaceMap)
    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    ans = stats.pearsonr(population_by_region, win_loss_by_region)[0]
    #print (ans)

   
    return ans

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nba_correlation():
    # YOUR CODE HERE
    replaceMap = {'Indianapolis Pacers':'Indiana Pacers',
                                  'Salt Lake Jazz':'Utah Jazz',
                                  'New York Nets':'Brooklyn Nets',
                                  'San Francisco Warriors':'Golden State Warriors',
                                  'Minneapolis Timberwolves':'Minnesota Timberwolves',
                                  'Oklahoma Thunder':'Oklahoma City Thunder'
                                 }
    #raise NotImplementedError()
    common_df = nba_df
    big4_df = cities['NBA']


#    cities.rename(columns={'Metropolitan area':'MA','Population (2016 est.)[8]':'Population'},inplace=True)
#    df= cities
#    regions=cities['MA']
#    regions = regions.str.replace('^(?P<ONE>.*)–.*$',lambda m: m.group('ONE')).str.replace('^(?P<ONE>.*), D\.C\.$',lambda m: m.group('ONE')).str.replace('^(?P<ONE>.*) Bay Area$',lambda m: m.group('ONE')).str.replace('^(?P<ONE>.*) Area$',lambda m: m.group('ONE')).str.replace('^(?P<ONE>.*) City$',lambda m: m.group('ONE'))

#    df = common_df
#    df = df.replace(r'^\s*$', np.NaN, regex=True).replace(r'^.*—.*$', np.NaN, regex=True).replace(r'^$', np.NaN, regex=True)
#    df = df.str.replace(r'^(?P<ONE>.*)\[.*\](?P<TWO>.*)$', lambda m: m.group('ONE') + m.group('TWO'), regex=True)
#    findf = df.str.findall('([A-Z0-9][0-9a-z]+ [A-Z0-9][0-9a-z]+|[A-Z0-9][0-9a-z]+)').apply(lambda x:pd.Series(x))
#    tdf = cities
#    tdf['regions']= regions
#    colname = ['team1','team2','team3']
#    tdf[[colname[x] for x in findf]] = pd.DataFrame({x:findf[x] for x in findf})
#    df= tdf[['regions','Population']]
#    df[[colname[x] for x in findf]] = tdf[[colname[x] for x in findf]]
#    df = pd.melt(df, id_vars=['regions','Population'], value_vars=[colname[x] for x in findf], value_name='team')
#    df=df.dropna()
#    df= df[['regions','Population','team']]
#    df['team']= df['regions'] + ' ' + df['team'] 
#    df['team'] = df['team'].replace(replaceMap)
#    df['Population']=df['Population'].apply(pd.to_numeric)
#    df1=df
#    population_by_region = df.groupby(by='regions').mean()['Population']
#    print(population_by_region)

#    repl = lambda m: m.group('ONE')
#    df = common_df
#    df = df[~ df['team'].str.contains('Division')][df['year']==2018]
#    df[['W','L']] = df[['W','L']].apply(pd.to_numeric)
#    df = pd.DataFrame({'team':df['team'].str.replace('^(?P<ONE>.*)\s\([0-9]+\)$',repl, regex=True).str.replace('^(?P<ONE>[^\*]+)\**$',repl, regex=True),'ratio':df['W']/(df['W']+ df['L'])})

#    group_df = df1.merge(df, how='left', on='team').groupby(by='regions').mean()
#    win_loss_by_region=group_df['ratio']

#    FalseMatrix = win_loss_by_region.notnull()
#    win_loss_by_region = win_loss_by_region[FalseMatrix].tolist()
#    population_by_region = population_by_region[FalseMatrix].tolist()

#    ans = stats.pearsonr(population_by_region, win_loss_by_region)[0]    
    
    population_by_region, win_loss_by_region = common_correlation(cities, big4_df, common_df, replaceMap)
    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"
    ans = stats.pearsonr(population_by_region, win_loss_by_region)[0] 
    #print(ans)


    return ans

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [3]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]



def mlb_correlation(): 
    # YOUR CODE HERE
    #raise NotImplementedError()
    

    replaceMap = {'Kansas Royals':'Kansas City Royals',
                                      'Tampa Rays':'Tampa Bay Rays',
                                      'Denver Rockies':'Colorado Rockies',
                                      'Minneapolis Twins':'Minnesota Twins',
                                      'Phoenix Diamondbacks':'Arizona Diamondbacks',
                                      'Dallas Rangers':'Texas Rangers',
                                      'San Francisco Athletics':'Oakland Athletics'
                                     }
        #
    population_by_region, win_loss_by_region = common_correlation(cities, cities['MLB'], mlb_df, replaceMap)
    ans = stats.pearsonr(population_by_region, win_loss_by_region)[0]
    #print(ans)    
    #population_by_region = [] # pass in metropolitan area population from cities
    #win_loss_by_region = [] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return ans

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [5]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]



def nfl_correlation(): 
    # YOUR CODE HERE
    #Tampa Buccaneers  Minnesota Vikings Charlotte Panthers Kansas Chiefs Arizona Cardinals Tennessee Titans
    replaceMap = {'Kansas Chiefs':'Kansas City Chiefs',
                                      'Tampa Buccaneers':'Tampa Bay Buccaneers',
                                      'Charlotte Panthers':'Carolina Panthers',
                                      'Minneapolis Vikings':'Minnesota Vikings',
                                      'Phoenix Cardinals':'Arizona Cardinals',
                                      'Nashville Titans':'Tennessee Titans',
                                      'Boston Patriots':'New England Patriots'
                                     }
        #
    population_by_region, win_loss_by_region = common_correlation(cities, cities['NFL'], nfl_df, replaceMap)
    ans = stats.pearsonr(population_by_region, win_loss_by_region)[0]
    print(ans)        
    #population_by_region = [] # pass in metropolitan area population from cities
    #win_loss_by_region = [] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return ans

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [87]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values